In [2]:
import torch

def swap_top_two_softmax(tensor):
    # Find the top two values along the softmax dimension
    top2_values, top2_indices = torch.topk(tensor, 2, dim=1)
    
    # Clone the original tensor so we can modify it
    swapped_tensor = tensor.clone()
    
    # Create a tensor of batch indices
    batch_indices = torch.arange(tensor.size(0))
    
    # Get the indices of the top two values
    max_indices = top2_indices[:, 0]  # Highest value indices
    second_max_indices = top2_indices[:, 1]  # Second highest value indices
    
    # Swap the values by indexing per batch
    swapped_tensor[batch_indices, max_indices] = top2_values[:, 1]  # Place second highest value at highest index
    swapped_tensor[batch_indices, second_max_indices] = top2_values[:, 0]  # Place highest value at second highest index
    
    return swapped_tensor

# Example usage
tensor = torch.tensor([[0.1, 0.3, 0.4, 0.2],
                       [0.5, 0.2, 0.1, 0.2]])
swapped_tensor = swap_top_two_softmax(tensor)
print(swapped_tensor)

tensor([[0.1000, 0.4000, 0.3000, 0.2000],
        [0.2000, 0.5000, 0.1000, 0.2000]])
